In [3]:
import os, datetime
import cv2
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import random
from pathlib import Path

time_str = (datetime.datetime.now()).strftime("%Y%m%d")
time_str = '20190507'
xml_dir = '/data/darknet/python/' + time_str + '_xml/'
SEED_XML_DIR = xml_dir
GENE_XML_DIR = SEED_XML_DIR


# RT:RightTop 
# LB:LeftBottom 
# bbox: [xmin, xax, ymin, ymax]
def IOU(bbox_a, bbox_b):
    '''
    W = min(A.RT.x, B.RT.x) - max(A.LB.x, B.LB.x) 
    H = min(A.RT.y, B.RT.y) - max(A.LB.y, B.LB.y) 
    if W <= 0 or H <= 0: 
        return 0 
    SA = (A.RT.x - A.LB.x) * (A.RT.y - A.LB.y) 
    SB = (B.RT.x - B.LB.x) * (B.RT.y - B.LB.y) 
    cross = W * H return cross/(SA + SB - cross)
    '''
    W = min(bbox_a[1], bbox_b[1]) - max(bbox_a[0], bbox_b[0]) 
    H = min(bbox_a[3], bbox_b[3]) - max(bbox_a[2], bbox_b[2]) 
    if W <= 0 or H <= 0: 
        return 0
    SA = (bbox_a[1] - bbox_a[0]) * (bbox_a[3] - bbox_a[2]) 
    SB = (bbox_b[1] - bbox_b[0]) * (bbox_b[3] - bbox_b[2])  
    cross = W * H 
    return cross/(SA + SB - cross)

def get_obj_from_xml(xml):
    in_file = open(xml_name)
    tree=ET.parse(in_file)
    root = tree.getroot()
    return [obj for obj in root.iter('object')]

def get_obj_from_image_file(file, bbox):
    img = cv2.imread(file)
    img_obj = img[int(bbox[2]):int(bbox[3]), int(bbox[0]):int(bbox[1])]
    return img_obj

def get_bboxes_from_etree(etree):
    root = etree.getroot()  
    objects = root.findall('object')
    bboxes = []
    for obj in objects:
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        bboxes.append(b)
    return bboxes

seed_xml_names = os.listdir(SEED_XML_DIR)
seed_xml_names.sort()

classes_ = ['bus', 'car', 'truck', 'motorbike', 'bicycle', 'person']
#classes_ = ['person']

for xml_name in seed_xml_names:    
    if(not xml_name.endswith('.xml')):
        continue
    if(not (int(xml_name[-15:-6]) >= 3010000)):
        continue   
    #img_data = cv2.imread(SEED_IMG_DIR + xml_name[:-3] + 'jpg')
    print(SEED_XML_DIR + xml_name)
    in_file = open(SEED_XML_DIR + xml_name)
    tree=ET.parse(in_file)

    root = tree.getroot()
    size = root.find('size')   
    boxes = []
    b = []  
    wrong_detection_objs = [[1820, 1920, 870, 1080]]

    array_object = root.findall('object')
    for obj in array_object:
        erro_obj_removed = False
        difficult = obj.find('difficult').text
        cls_ = obj.find('name').text
        if cls_ not in classes_ or int(difficult)==1:
            continue
        xmlbox = obj.find('bndbox')
        b = [float(xmlbox.find('xmin').text), float(xmlbox.find('xmax').text), float(xmlbox.find('ymin').text), float(xmlbox.find('ymax').text)]
        for erro_bbox in wrong_detection_objs:
            if(IOU(erro_bbox, b) > 0.7):
                print(IOU(erro_bbox, b))
                print('removed')
                root.remove(obj)
                erro_obj_removed = True
                break
        if(erro_obj_removed):
            continue
        '''
        #filter size which smaller than 16X16 fit for SSD300
        bbox_width = b[1] - b[0]
        bbox_heigt = b[3] - b[2]
        if(not(bbox_width > 16 or bbox_heigt > 16)):
            print('remove:{0}:{1}'.format(bbox_width, bbox_heigt))
            root.remove(obj)
            continue 
        else:
            print('add:{0}:{1}'.format(bbox_width, bbox_heigt))
        '''    
    tree.write(GENE_XML_DIR + xml_name)

/data/darknet/python/20190507_xml/003010000.0.xml
/data/darknet/python/20190507_xml/003010002.0.xml
/data/darknet/python/20190507_xml/003010004.0.xml
/data/darknet/python/20190507_xml/003010005.0.xml
/data/darknet/python/20190507_xml/003010007.0.xml
/data/darknet/python/20190507_xml/003010009.0.xml
0.9428357930132636
removed
/data/darknet/python/20190507_xml/003010010.0.xml
/data/darknet/python/20190507_xml/003010011.0.xml
/data/darknet/python/20190507_xml/003010012.0.xml
/data/darknet/python/20190507_xml/003010013.0.xml
/data/darknet/python/20190507_xml/003010014.0.xml
/data/darknet/python/20190507_xml/003010015.0.xml
/data/darknet/python/20190507_xml/003010016.0.xml
/data/darknet/python/20190507_xml/003010017.0.xml
/data/darknet/python/20190507_xml/003010019.0.xml
/data/darknet/python/20190507_xml/003010020.0.xml
/data/darknet/python/20190507_xml/003010021.0.xml
/data/darknet/python/20190507_xml/003010022.0.xml
/data/darknet/python/20190507_xml/003010023.0.xml
/data/darknet/python/20

/data/darknet/python/20190507_xml/003010228.0.xml
/data/darknet/python/20190507_xml/003010229.0.xml
/data/darknet/python/20190507_xml/003010230.0.xml
/data/darknet/python/20190507_xml/003010232.0.xml
/data/darknet/python/20190507_xml/003010233.0.xml
/data/darknet/python/20190507_xml/003010235.0.xml
/data/darknet/python/20190507_xml/003010236.0.xml
/data/darknet/python/20190507_xml/003010237.0.xml
/data/darknet/python/20190507_xml/003010239.0.xml
/data/darknet/python/20190507_xml/003010240.0.xml
/data/darknet/python/20190507_xml/003010241.0.xml
/data/darknet/python/20190507_xml/003010242.0.xml
/data/darknet/python/20190507_xml/003010243.0.xml
/data/darknet/python/20190507_xml/003010246.0.xml
/data/darknet/python/20190507_xml/003010247.0.xml
/data/darknet/python/20190507_xml/003010248.0.xml
/data/darknet/python/20190507_xml/003010249.0.xml
/data/darknet/python/20190507_xml/003010250.0.xml
/data/darknet/python/20190507_xml/003010252.0.xml
/data/darknet/python/20190507_xml/003010253.0.xml


/data/darknet/python/20190507_xml/003010455.0.xml
/data/darknet/python/20190507_xml/003010456.0.xml
/data/darknet/python/20190507_xml/003010457.0.xml
/data/darknet/python/20190507_xml/003010458.0.xml
0.9615384615384616
removed
/data/darknet/python/20190507_xml/003010459.0.xml
/data/darknet/python/20190507_xml/003010460.0.xml
/data/darknet/python/20190507_xml/003010461.0.xml
/data/darknet/python/20190507_xml/003010462.0.xml
/data/darknet/python/20190507_xml/003010463.0.xml
/data/darknet/python/20190507_xml/003010464.0.xml
/data/darknet/python/20190507_xml/003010465.0.xml
/data/darknet/python/20190507_xml/003010467.0.xml
/data/darknet/python/20190507_xml/003010469.0.xml
/data/darknet/python/20190507_xml/003010470.0.xml
/data/darknet/python/20190507_xml/003010472.0.xml
/data/darknet/python/20190507_xml/003010473.0.xml
/data/darknet/python/20190507_xml/003010474.0.xml
/data/darknet/python/20190507_xml/003010475.0.xml
/data/darknet/python/20190507_xml/003010476.0.xml
/data/darknet/python/20

/data/darknet/python/20190507_xml/003010888.0.xml
/data/darknet/python/20190507_xml/003010890.0.xml
/data/darknet/python/20190507_xml/003010891.0.xml
/data/darknet/python/20190507_xml/003010892.0.xml
/data/darknet/python/20190507_xml/003010893.0.xml
/data/darknet/python/20190507_xml/003010894.0.xml
/data/darknet/python/20190507_xml/003010896.0.xml
/data/darknet/python/20190507_xml/003010897.0.xml
/data/darknet/python/20190507_xml/003010898.0.xml
/data/darknet/python/20190507_xml/003010900.0.xml
/data/darknet/python/20190507_xml/003010901.0.xml
/data/darknet/python/20190507_xml/003010903.0.xml
0.9519230769230769
removed
/data/darknet/python/20190507_xml/003010904.0.xml
0.8085446934252647
removed
/data/darknet/python/20190507_xml/003010905.0.xml
/data/darknet/python/20190507_xml/003010906.0.xml
/data/darknet/python/20190507_xml/003010909.0.xml
/data/darknet/python/20190507_xml/003010910.0.xml
/data/darknet/python/20190507_xml/003010913.0.xml
/data/darknet/python/20190507_xml/003010914.0.

/data/darknet/python/20190507_xml/003011124.0.xml
/data/darknet/python/20190507_xml/003011126.0.xml
0.8091346153846154
removed
/data/darknet/python/20190507_xml/003011127.0.xml
0.8050677666470242
removed
/data/darknet/python/20190507_xml/003011132.0.xml
/data/darknet/python/20190507_xml/003011133.0.xml
/data/darknet/python/20190507_xml/003011134.0.xml
/data/darknet/python/20190507_xml/003011137.0.xml
/data/darknet/python/20190507_xml/003011138.0.xml
/data/darknet/python/20190507_xml/003011139.0.xml
/data/darknet/python/20190507_xml/003011141.0.xml
/data/darknet/python/20190507_xml/003011142.0.xml
/data/darknet/python/20190507_xml/003011143.0.xml
/data/darknet/python/20190507_xml/003011144.0.xml
/data/darknet/python/20190507_xml/003011145.0.xml
/data/darknet/python/20190507_xml/003011146.0.xml
/data/darknet/python/20190507_xml/003011147.0.xml
/data/darknet/python/20190507_xml/003011148.0.xml
/data/darknet/python/20190507_xml/003011149.0.xml
/data/darknet/python/20190507_xml/003011150.0.

/data/darknet/python/20190507_xml/003011536.0.xml
/data/darknet/python/20190507_xml/003011537.0.xml
/data/darknet/python/20190507_xml/003011538.0.xml
0.8285714285714286
removed
/data/darknet/python/20190507_xml/003011539.0.xml
0.7899176541474192
removed
/data/darknet/python/20190507_xml/003011540.0.xml
/data/darknet/python/20190507_xml/003011541.0.xml
/data/darknet/python/20190507_xml/003011542.0.xml
/data/darknet/python/20190507_xml/003011543.0.xml
0.7950566426364573
removed
/data/darknet/python/20190507_xml/003011544.0.xml
/data/darknet/python/20190507_xml/003011545.0.xml
/data/darknet/python/20190507_xml/003011546.0.xml
/data/darknet/python/20190507_xml/003011547.0.xml
/data/darknet/python/20190507_xml/003011548.0.xml
/data/darknet/python/20190507_xml/003011549.0.xml
/data/darknet/python/20190507_xml/003011550.0.xml
/data/darknet/python/20190507_xml/003011552.0.xml
/data/darknet/python/20190507_xml/003011553.0.xml
/data/darknet/python/20190507_xml/003011554.0.xml
/data/darknet/pytho

/data/darknet/python/20190507_xml/003011806.0.xml
0.9519230769230769
removed
/data/darknet/python/20190507_xml/003011807.0.xml
/data/darknet/python/20190507_xml/003011808.0.xml
0.8085446934252647
removed
/data/darknet/python/20190507_xml/003011809.0.xml
0.847612958226769
removed
/data/darknet/python/20190507_xml/003011810.0.xml
/data/darknet/python/20190507_xml/003011811.0.xml
/data/darknet/python/20190507_xml/003011812.0.xml
/data/darknet/python/20190507_xml/003011813.0.xml
/data/darknet/python/20190507_xml/003011814.0.xml
/data/darknet/python/20190507_xml/003011818.0.xml
/data/darknet/python/20190507_xml/003011819.0.xml
/data/darknet/python/20190507_xml/003011820.0.xml
/data/darknet/python/20190507_xml/003011823.0.xml
/data/darknet/python/20190507_xml/003011825.0.xml
/data/darknet/python/20190507_xml/003011827.0.xml
/data/darknet/python/20190507_xml/003011828.0.xml
/data/darknet/python/20190507_xml/003011829.0.xml
/data/darknet/python/20190507_xml/003011832.0.xml
/data/darknet/python

/data/darknet/python/20190507_xml/003012076.0.xml
/data/darknet/python/20190507_xml/003012077.0.xml
/data/darknet/python/20190507_xml/003012078.0.xml
/data/darknet/python/20190507_xml/003012079.0.xml
/data/darknet/python/20190507_xml/003012080.0.xml
/data/darknet/python/20190507_xml/003012082.0.xml
/data/darknet/python/20190507_xml/003012084.0.xml
/data/darknet/python/20190507_xml/003012086.0.xml
/data/darknet/python/20190507_xml/003012087.0.xml
/data/darknet/python/20190507_xml/003012088.0.xml
/data/darknet/python/20190507_xml/003012089.0.xml
/data/darknet/python/20190507_xml/003012090.0.xml
/data/darknet/python/20190507_xml/003012091.0.xml
/data/darknet/python/20190507_xml/003012092.0.xml
/data/darknet/python/20190507_xml/003012094.0.xml
/data/darknet/python/20190507_xml/003012095.0.xml
/data/darknet/python/20190507_xml/003012098.0.xml
/data/darknet/python/20190507_xml/003012099.0.xml
/data/darknet/python/20190507_xml/003012101.0.xml
/data/darknet/python/20190507_xml/003012103.0.xml
